# SHAKTI FEniCS code

The code requires [FEniCSx](https://fenicsproject.org). 

In [1]:
%load_ext autoreload
%autoreload 2
import sys, time
sys.path.insert(0, '../source')

Set model options in main.py and then run the code: 

In [2]:
!rm -r /Users/agstubbl/Desktop/shakti-fenics/results/cooke2

rm: /Users/agstubbl/Desktop/shakti-fenics/results/cooke2: No such file or directory


In [3]:
!mpirun -np 4 python3 ../source/main.py setup_cooke2

Info    : Reading '../meshes/cooke2.msh'...
Info    : 301 entities
Info    : 1307 nodes
Info    : 2462 elements
Info    : Done reading '../meshes/cooke2.msh'

refining mesh...

